In [1]:
import redis
import time

In [2]:
redis_host = 'localhost'
redis_port = 6379
redis_password = 'mypassword'

In [3]:
# later create the object with account details when vpn works.
DB = redis.Redis(host=redis_host, port=redis_port, db=0, charset='utf-8',decode_responses=True, password=redis_password)

In [4]:
DB.set("city", "Tomsk")
print(DB.get("city"))

Tomsk


In [5]:
# Create DB object here
# DBcloud = redis.Redis()


In [6]:
# DBcloud.set("University", "TSU")
# print(DBcloud.get("University"))

In [7]:
DB.set("time", int(time.time()), 20)
print(DB.get("time"))

1719649278


In [8]:
DB.set("index_field", 1)
print(f"index_field: {DB.get('index_field')}")

index_field: 1


In [9]:
# let's increase the index_field by a 1
DB.incr('index_field')
print(f"increased index field: {DB.get('index_field')}")

increased index field: 2


In [10]:
# let's increase it by 10
DB.incrby("index_field", 10)
print(f"new index field increased by 10 is: {DB.get('index_field')}")

new index field increased by 10 is: 12


In [11]:
# reduce it by  5
DB.decrby("index_field", 5)
print(f"Decreased index field: {DB.get('index_field')}")

Decreased index field: 7


## List

In [12]:
DB.lpush("tomsk_weather", "white winter")

26

In [13]:
print(f"Weather in tomsk when i arrived: {DB.lrange('tomsk_weather', 0, -1)}")

Weather in tomsk when i arrived: ['white winter', 'white winter', 'white winter', 'white winter', 'green winter', 'semi-summer', 'spring', 'green winter', 'semi-summer', 'spring', 'green winter', 'semi-summer', 'spring', 'green winter', 'semi-summer', 'spring', 'green winter', 'semi-summer', 'spring', 'green winter', 'semi-summer', 'spring', 'green winter', 'semi-summer', 'spring', 'green winter']


In [15]:
# add some other weather in tomsk
DB.rpush('tomsk_weather', 'green winter', 'semi-summer', 'spring')
print(f"Weather Seasons in tomsk after white winter: {DB.lrange('tomsk_weather',0, -1)}")
print(f"length: {DB.llen('tomsk_weather')}")

Weather Seasons in tomsk after white winter: ['white winter', 'white winter', 'white winter', 'white winter', 'green winter', 'semi-summer', 'spring', 'green winter', 'semi-summer', 'spring', 'green winter', 'semi-summer', 'spring', 'green winter', 'semi-summer', 'spring', 'green winter', 'semi-summer', 'spring', 'green winter', 'semi-summer', 'spring', 'green winter', 'semi-summer', 'spring', 'green winter', 'green winter', 'green winter']


In [16]:
# let's remove some items 
DB.lrem('tomsk_weather', 1, 'white_winter')
print(f"Weather Seasons in tomsk after white winter: {DB.lrange('tomsk_weather',0, -1)}")

Weather Seasons in tomsk after white winter: ['white winter', 'white winter', 'white winter', 'white winter', 'green winter', 'semi-summer', 'spring', 'green winter', 'semi-summer', 'spring', 'green winter', 'semi-summer', 'spring', 'green winter', 'semi-summer', 'spring', 'green winter', 'semi-summer', 'spring', 'green winter', 'semi-summer', 'spring', 'green winter', 'semi-summer', 'spring', 'green winter', 'green winter', 'green winter']


One major disadvantage of list is that it can contain duplicates. If our use case problem requires us to have unique entries, then it would be beneficial to use a set instead

## Set


Now i am going to create a set to store the weather seasons in tomsk. 

In [17]:
DB.sadd("tomsk_seasons", 'white winter')
print(f"Tomsk's first season is: {DB.smembers('tomsk_seasons')}")

Tomsk's first season is: {'white winter'}


In [18]:
# Let's add the green winter season
DB.sadd("tomsk_seasons", "green winter")
print(f"Tomsk's seasons: {DB.smembers('tomsk_seasons')}")

Tomsk's seasons: {'white winter', 'green winter'}


In [22]:
# let's add multiple seasons to it
DB.sadd("tomsk_seasons", "summer", "spring", "Autumn")
print(f"Tomsk's Seasons: {DB.smembers('tomsk_seasons')}")

Tomsk's Seasons: {'white winter', 'green winter', 'summer', 'Autumn', 'spring'}


In [26]:
# Seasons in Nigeria
DB.sadd("nigeria_seasons", "Raining", "Dry")
print(f"Nigeria's Seasons: {DB.smembers('nigeria_seasons')}")
print()
print("Let's find the union of the seasons in nigeria and tomsk")
print()
print(f"The Union is : {DB.sunion('tomsk_seasons', 'nigeria_seasons')}")
print()
print("The difference between the two nigeria and tomsk")
print()
print(f"The difference is : {DB.sdiff('nigeria_seasons', 'tomsk_seasons')}")

Nigeria's Seasons: {'Dry', 'Raining'}

Let's find the union of the seasons in nigeria and tomsk

The Union is : {'white winter', 'Raining', 'green winter', 'summer', 'Autumn', 'Dry', 'spring'}

The difference between the two nigeria and tomsk

The difference is : {'Dry', 'Raining'}


### Adding items to a sorted set with the method zadd()

A unique thing about  working with sets is that it is a unique collection of members. we can't have duplicates in a set. However, we still have a little drawback. Sets are not ordered.
Now we're going to create an ordered set in the next section


In [27]:
DB.zadd('top_db', {
    'MySQL':1,
    'Microsoft SQL Server': 2,
    'Oracle': 3
})

print(f"top_db: {DB.zrange('top_db', 0, -1)}")

top_db: ['MySQL', 'Microsoft SQL Server', 'Oracle']


In [34]:
# Add a new item to the set at a specific position
DB.zadd('top_db', {'PostgreSQL':3})
DB.zadd('top_db', {'MongoDB':2})
DB.zadd('top_db', {'Redis':1})
print("Let's see what our ordered set looks like")
print()
print(f"Here's our set: {DB.zrange('top_db', 0,-1)}")
print()
print("What is the length of our set? ")
print()
print(f"Length: {DB.zcard('top_db')}")

Let's see what our ordered set looks like

Here's our set: ['MySQL', 'Redis', 'Microsoft SQL Server', 'MongoDB', 'Oracle', 'PostgreSQL']
Length: 6


## Hashmaps
Hashmaps is what we call Dictionary in python

In [43]:
products = {
    'name': "Oracle",
    'description': 'Database',
    'rating': '5'
}

In [50]:
DB.hmset('products_rating', products)
print(f"Here's our products, a description and the rating: {DB.hgetall('products_rating')}")
print()
print("Let's get the keys in the hashmap")
print()
print(f"The keys are: {DB.hkeys('products_rating')}")
print()
print("For the values we have")
print(f"Values of the hashmap: {DB.hvals('products_rating')}")

Here's our products, a description and the rating: {'name': 'Oracle', 'description': 'Database', 'rating': '5'}

Let's get the keys in the hashmap

The keys are: ['name', 'description', 'rating']

For the values we have
Values of the hashmap: ['Oracle', 'Database', '5']


/var/folders/y5/5kc5fqvn2tncnl_868zg4zhr0000gn/T/ipykernel_70759/130199670.py:1: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  DB.hmset('products_rating', products)
